<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Morpheme analyzer
*   [Reference](https://soohee410.github.io/compare_tagger)
*   [Install](https://sanghyu.tistory.com/170)

In [ ]:
# okt, komoran, kkma
# install konlpy (okt, komoran, kkma)
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

# mecab (take a long time)
# set env
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

# install konlpy (mecab)
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# Error Solution

1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings' [(JVM)](https://gyulogs.tistory.com/130)
2. NameError: name 'Tagger' is not defined [(Mecab)](https://sosomemo.tistory.com/31)
3. ParserError: Error tokenizing data. C error [(Pandas)](https://mskim8717.tistory.com/82)



In [ ]:
""" 
ERROR 1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings'
- Solution: /usr/local/lib/python3.7/dist-packages/konlpy/jvm.py, 67 line (convertStrings=True) comments processing and save jvm.py before import pakage
"""

""" 
ERROR 2. NameError: name 'Tagger' is not defined 
- Solution: Execute mecab.sh script (under code excute)
"""
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# Import
*   [KoNLPy Reperence](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)



In [ ]:
# import for MechineLearning
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# import Morpheme analyzer
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from konlpy.utils import pprint

# import etc
import time

%cd /content/drive/My Drive/DeepLearning/AI_KKH

#Grobal Variable

In [ ]:
AI_TARGET_NAME = "프로젝트 김대엽 클럽장"

# Funtion

In [ ]:
"""
Function about time comparison

def pos_times(taggers, tagger_name, texts) :
    time_list = []
    for tagger in taggers :
        print(type(tagger))
        
        t1 = time.time()
        for text in texts :
            pprint(tagger.pos(text))
        
        print("###################################")
        t2 = time.time()
        time_list.append(t2 - t1)

    plt.figure(figsize=(10,8))
    plt.bar(tagger_name, time_list, color=(0.4,0.7,0.5))
    plt.title('Learning Time with .pos', fontsize=17)
    plt.ylabel('total seconds')
"""

# Extract reply time (minute)
def extract_replytime(content) :
    end_idx = content.rfind(']')
    start_idx = content[:end_idx].rfind('[') + 1

    replytime = content[start_idx : end_idx]

    if replytime[:2] == "오전" :
        replytime = (int(replytime[2:replytime.index(":")]) * 60) + int(replytime[replytime.index(":") + 1 :])
    else :
        replytime = 60 * 12 + (int(replytime[2:replytime.index(":")]) * 60) + int(replytime[replytime.index(":") + 1 :])

    return replytime

# Comparison about reply time
def compare_replytime(pre_text, cur_text) :
    pre_time = extract_replytime(pre_text)
    cur_time = extract_replytime(cur_text)

    result = False
    if cur_time - pre_time >= 10 : result = True

    return result

# Extract Data about kakao talk's content
def extract_data(x_list, y_list) :
    tmp_X_list = []
    tmp_Y_list = []

    before_respondent = ""
    for text in test.values :
        text = text.tolist()
        for t in text : 
            if ":" in t and before_respondent != "" :
                cur_respondent = t[1:t.index("]")]
                if compare_replytime(text[text.index(t) - 1], t) or cur_respondent != before_respondent :
                    if len(tmp_X_list) != 0 : x_list.append(tmp_X_list)
                    if len(tmp_Y_list) != 0 : y_list.append(tmp_Y_list)

                    tmp_X_list = []
                    tmp_Y_list = []
            if AI_TARGET_NAME in t :
                ptext = t[(t.rfind(']') + 2) : len(t)]
                if ptext.find("http") == -1 and ptext != "사진" and ptext != "" : 
                    before_respondent = AI_TARGET_NAME
                    tmp_Y_list.append(ptext)
            elif "]" in t :
                ptext = t[(t.rfind(']') + 2) : len(t)]
                if ptext.find("http") == -1 and ptext != "사진" and ptext != "" : 
                    before_respondent = cur_respondent
                    tmp_X_list.append(ptext)

# Auto calibrate spacing function
def calibrate_spcaing(input_list) :
    for idx in range(0, len(input_list)) :
        for text in input_list[idx] :
            analyzedRes = mecab.pos(text)
            for mecabR in analyzedRes :
                if mecabR[1] == "MAG" :
                    try :
                        startIdx = text.index(mecabR[0])
                        if len(mecabR[0]) == 1 :
                            if text[startIdx + 1] != " " and text[startIdx + 1].isalnum() : 
                                if text[startIdx + 2] != " " :
                                    repairIdx = input_list[idx].index(text)
                                    input_list[idx][repairIdx] = text[:startIdx] + text[startIdx] + " " + text[startIdx + 1 :]
                        else : 
                            if text[startIdx + len(mecabR[0])] != " " and text[startIdx + len(mecabR[0])].isalnum() : 
                                if text[startIdx + len(mecabR[0]) + 1] != " " :
                                    repairIdx = input_list[idx].index(text)
                                    input_list[idx][repairIdx] = text[:startIdx] + text[startIdx : startIdx + len(mecabR[0])] + " " + text[startIdx + len(mecabR[0]) :]
                    except IndexError :
                        continue

# Extraction 1 word by 1 content
def extract_1word(input_list) :
    for idx in range(0, len(input_list)) : 
        input_list[idx] = input_list[idx][0]

# If x and y size different, equalize that
def equalize_size(x, y) :
    if len(x) > len(y) : x = x[0 : len(y)]
    elif len(y) > len(x) : y = y[0 : len(x)]

    return x, y

#Main

In [ ]:
"""
ranking about time spent morpheme analyzing (The fastest is number one.)

1. Mecab
2. Komoran
3. Okt
4. Kkma

Mecab is faster than Kkma about 30~40 times
Mecab is faster than Okt about 10 times
Mecab is faster than Komoran about 5 times
"""

# Early versions use okt and macab.
mecab = Mecab()
okt = Okt()

""" 
ERROR 3. ParserError: Error tokenizing data. C error 
- Solution: add code in read_csv = , sep='\t'
"""
test = pd.read_csv('KKH_20200129~20210707.txt', sep='\t')

# 1. Data extraction
x_list = []
y_list = []
extract_data(x_list, y_list)

# 2. Space cailbrating
calibrate_spcaing(x_list)
calibrate_spcaing(y_list)

# 3. Data manufacturing
# Early version is used y_list[][0] and x_list[][0] (1 word) by learning model
extract_1word(x_list)
extract_1word(y_list)
# Size equalize
x_list, y_list = equalize_size(x_list, y_list)
# One-hot encoding
"""
# 4. Modeling
model = Sequential()
"""